In [6]:
import os
import pandas as pd
import boto3
from dotenv import load_dotenv
from io import StringIO

In [7]:
load_dotenv()

access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = os.getenv('AWS_BUCKET_NAME')
region = os.getenv('AWS_REGION')

input_key = 'raw_sales_data.csv'
output_key = 'processed_electronics_sales.csv'

s3 = boto3.client(
    's3',
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name=region
)

In [8]:
# Extract from S3
obj = s3.get_object(Bucket=bucket_name, Key=input_key)
raw_data = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8')))
raw_data.head()

,sale_id,product_id,product_name,category,quantity_sold,price_per_unit,sale_date,region
0,1,101,Laptop Pro,Electronics,5,1200.0,2023-01-15,North
1,2,102,Desktop Max,Electronics,2,1500.0,2023-01-16,North
2,3,201,Office Chair,Furniture,10,150.0,2023-01-17,South
3,4,103,Wireless Mouse,Electronics,25,25.0,2023-01-18,West
4,5,301,Python Programming Book,Books,50,45.0,2023-01-18,East


In [10]:
# Transform

# Filter only Electronics category
filtered_data = raw_data[raw_data['category'] == 'Electronics'].copy()

# Add total_revenue column
filtered_data['total_revenue'] = filtered_data['quantity_sold'] * filtered_data['price_per_unit']

# Format sale_date
filtered_data.loc[:, 'sale_date'] = pd.to_datetime(filtered_data['sale_date']).dt.strftime('%Y-%m-%d')

# Reorder columns
filtered_data = filtered_data[['sale_id', 'product_name', 'category', 'total_revenue', 'sale_date', 'region']]
filtered_data.head()

,sale_id,product_name,category,total_revenue,sale_date,region
0,1,Laptop Pro,Electronics,6000.0,2023-01-15,North
1,2,Desktop Max,Electronics,3000.0,2023-01-16,North
3,4,Wireless Mouse,Electronics,625.0,2023-01-18,West
5,6,Laptop Pro,Electronics,3600.0,2023-01-19,South
7,8,Keyboard Pro,Electronics,1125.0,2023-01-20,East


In [ ]:
# Save the transformed data to a new CSV file
csv_buffer = StringIO()
filtered_data.to_csv(csv_buffer, index=False)

s3.put_object(Bucket=bucket_name, Key=output_key, Body=csv_buffer.getvalue())
print(f"Processed data saved to {output_key} in S3. ")

Processed data saved to processed_electronics_sales.csv in S3 bucket 
